In [ ]:
# 功能描述：挂载google drive，点击右侧的 挂载云端硬盘 按钮可以避免重复验证过程
import os
from google.colab import drive
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')

In [ ]:
# 自动设置和运行 frpc 客户端
import os
import subprocess

# 创建代码目录
code_dir = '/content/drive/MyDrive/code'
mcp_frp_dir = f'{code_dir}/mcp-frp'

# 创建目录
os.makedirs(code_dir, exist_ok=True)
print(f"确保目录存在: {code_dir}")

# 检查 mcp-frp 是否存在，不存在则下载，存在则更新
if not os.path.exists(mcp_frp_dir):
    print("mcp-frp 目录不存在，正在从 GitHub 下载...")
    os.chdir(code_dir)
    result = subprocess.run(['git', 'clone', 'https://github.com/eatcosmos/mcp-frp.git'], 
                          capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ mcp-frp 下载成功")
    else:
        print(f"❌ 下载失败: {result.stderr}")
        raise Exception("无法下载 mcp-frp")
else:
    print("✅ mcp-frp 目录已存在，正在拉取最新版本...")
    os.chdir(mcp_frp_dir)
    result = subprocess.run(['git', 'pull'], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ 代码更新成功")
        if "Already up to date" in result.stdout:
            print("ℹ️ 代码已经是最新版本")
        else:
            print("📦 已拉取最新更改")
    else:
        print(f"⚠️ 更新失败，继续使用现有版本: {result.stderr}")
        # 即使更新失败也继续执行，使用现有版本

# 设置环境变量
from google.colab import userdata
try:
    server_ip = userdata.get('FRPS_SERVER_IP')
    os.environ['FRPS_SERVER_IP'] = server_ip
    print(f"✅ 设置服务器 IP: {server_ip}")
except:
    print("❌ 请在 colab secrets 中设置 FRPS_SERVER_IP")
    raise Exception("需要设置 FRPS_SERVER_IP")

# 运行 frpc 客户端
print("正在启动 frpc 客户端...")
os.chdir(mcp_frp_dir)
subprocess.run(['chmod', '+x', 'run-frpc.sh'])

# 后台运行 frpc
result = subprocess.run(['./run-frpc.sh', '-d'], capture_output=True, text=True)
print("STDOUT:", result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)

if result.returncode == 0:
    print("✅ frpc 客户端已启动")
    print(f"🔗 SSH 连接: ssh -p 7502 root@{server_ip}")
    print("💡 提示: 如果是 colab 环境，用户名通常是 root")
else:
    print("❌ frpc 启动失败")


In [ ]:
# 管理 frpc 服务
import subprocess
import os

def stop_frpc():
    """停止 frpc 服务"""
    result = subprocess.run(['pkill', '-f', 'frpc -c frpc.toml'], 
                          capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ frpc 服务已停止")
    else:
        print("ℹ️ 没有找到运行中的 frpc 服务")

def check_frpc_status():
    """检查 frpc 服务状态"""
    result = subprocess.run(['pgrep', '-f', 'frpc -c frpc.toml'], 
                          capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✅ frpc 正在运行，PID: {result.stdout.strip()}")
        
        # 检查日志
        log_file = '/content/drive/MyDrive/code/mcp-frp/frp_0.63.0_linux_amd64/frpc.log'
        if os.path.exists(log_file):
            print("\n📋 最近的日志:")
            result = subprocess.run(['tail', '-5', log_file], 
                                  capture_output=True, text=True)
            print(result.stdout)
    else:
        print("❌ frpc 没有在运行")

def restart_frpc():
    """重启 frpc 服务"""
    print("正在重启 frpc...")
    stop_frpc()
    
    # 设置环境变量
    from google.colab import userdata
    try:
        server_ip = userdata.get('FRPS_SERVER_IP')
        os.environ['FRPS_SERVER_IP'] = server_ip
    except:
        print("❌ 请在 colab secrets 中设置 FRPS_SERVER_IP")
        return
    
    # 重新启动
    os.chdir('/content/drive/MyDrive/code/mcp-frp')
    result = subprocess.run(['./run-frpc.sh', '-d'], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ frpc 重启成功")
    else:
        print("❌ frpc 重启失败")
        print(result.stderr)

# 检查当前状态
print("=== frpc 服务状态 ===")
check_frpc_status()

print("\n=== 可用命令 ===")
print("stop_frpc()      # 停止服务")
print("check_frpc_status()  # 检查状态")
print("restart_frpc()   # 重启服务")
